# Robot experimentations
This notebook gives the necessary functions to experiment with the benchtop model of the Doedicurus tail club. 
The next cell gives all the necessary functions and sets up the motors.

In [1]:
from dynio import *
import time
dxl_io = dxl.DynamixelIO('COM3') #initialise USB port
body_motor = dxl_io.new_mx64(1, 1)  # MX-64 protocol 1 with ID 2
tail_motor = dxl_io.new_mx64(2, 1)  # MX-64 protocol 2 with ID 3
tail_motor.torque_enable()
body_motor.torque_enable()

tail_angle = []; tail_load = []; body_load = [];body_angle = []

def angle_to_motor_position(angle):
    """Transform an angle value in degrees to the motor position
    Args:
        angle (float): Angle in degrees
    Returns:
        int: Motor position
    """
    return int(2048 + 2048*angle/180)

zero_pos = angle_to_motor_position(0)
tail_min_pos = angle_to_motor_position(-45)
tail_max_pos = angle_to_motor_position(45)

body_motor.set_velocity(50)
tail_motor.set_velocity(50)
tail_motor.set_position(zero_pos)
body_motor.set_position(zero_pos)

def instance(body_velocity = 150, tail_velocity = 250, body_final_pos = 30, blow_time = 0.4, body_rot_offset = 0, max_time = 3, factor = 1):
    """This function performs an instance of experimentation for the robot.
    Args:
        body_velocity (int, optional): Body velocity in rev/min. Defaults to 150.
        tail_velocity (int, optional): Tail velocity in rev/min. Defaults to 250.
        body_final_pos (int, optional): Body final position in degrees. Defaults to 30.
        blow_time (float, optional): Swing offset in seconds. Defaults to 0.4.
        body_rot_offset (int, optional): Body rotation offset in seconds. Defaults to 0.
        max_time (int, optional): Time of the instance in seconds. Defaults to 3.
        factor (int, optional): Factor of power of preliminary swing. Defaults to 1.

    Returns:
        list: Angle and current load of both motors
    """
    body_max_pos = angle_to_motor_position(body_final_pos/2)
    body_min_pos = angle_to_motor_position(-body_final_pos/2)
    body_motor.set_velocity(int(body_velocity*factor))
    tail_motor.set_velocity(int(tail_velocity*factor))
    tail_motor.set_position(tail_min_pos)
    body_motor.set_position(body_min_pos)
    start_time = time.time()
    timer = 0
    while timer < max_time:
        timer = time.time() - start_time
        if timer> blow_time:
            body_motor.set_velocity(body_velocity)
            tail_motor.set_velocity(tail_velocity)
            if body_rot_offset>0:
                tail_motor.set_position(tail_max_pos)
            else:
                body_motor.set_position(body_max_pos)

        if timer> blow_time + abs(body_rot_offset):
            if body_rot_offset> 0:
                body_motor.set_position(body_max_pos)
            else:
                tail_motor.set_position(tail_max_pos)
        tail_angle.append(tail_motor.get_angle())
        tail_load.append(tail_motor.get_current())
        body_angle.append(body_motor.get_angle())
        body_load.append(body_motor.get_current())

    body_motor.set_position(zero_pos)
    tail_motor.set_position(zero_pos)
    return tail_angle, tail_load, body_load, body_angle

The following cell experiments with various parameters choice.

In [2]:
for i in range(0, 9):
    blow_time = 0.1*i
    instance(blow_time = blow_time)
    time.sleep(3)

for i in range(0, 7):
    instance(body_final_pos = i*10)
    time.sleep(3)

for i in range(0,9):
    body_rot_offset = -0.4 +0.1*i
    instance(body_rot_offset = body_rot_offset)
    time.sleep(3)

for i in range(0,10):
    body_velocity = 30+30*i
    instance(body_velocity = body_velocity)
    time.sleep(3)
    
for i in range(0,10):
    tail_velocity = 50+50*i
    instance(tail_velocity=tail_velocity)
    time.sleep(3)

for i in range(0,9):
    factor = 0.1*i+0.1
    instance(factor=factor)
    time.sleep(3)

instance(0,0)

[RxPacketError] Out of range error!
